In [ ]:
# - CODE IN THIS CELL TAKEN FROM 'py_markdown_table' LIBRARY DOCUMENTATION AND EDITED SLIGHTLY - 
# https://pypi.org/project/py-markdown-table/ 
from py_markdown_table.markdown_table import markdown_table

data = [
    {
        "" : "Person 1",
        "PBI 1": "5h 15 m", #also works with numerical values
        "PBI 2": "0 h",
        "PBI 3": "2 h 50 m"
    },
    {
        "" : "Person 2",
        "PBI 1": "6h 45 m", 
        "PBI 2": "0 h",
        "PBI 3": "8 h"
    },
    {
        "" : "Person 3",
        "PBI 1": "0 h", 
        "PBI 2": "10 h 30 m",
        "PBI 3": "2 h 45 m"
    }
]


markdown = markdown_table(data).set_params(padding_width = 2, quote=False, row_sep = 'markdown').get_markdown()
print(markdown)


|          |  PBI 1  |   PBI 2   |   PBI 3  |
|----------|---------|-----------|----------|
| Person 1 | 5h 15 m |    0 h    | 2 h 50 m |
| Person 2 | 6h 45 m |    0 h    |    8 h   |
| Person 3 |   0 h   | 10 h 30 m | 2 h 45 m |

In [ ]:
#Set the url to be queried and set the personal access token for authentication
GITLAB_API_BASE_URL = "https://gitlab.com/api/graphql" 
token = input("Enter access token: ")
#The query to be queried at the specified endpoint
test_query = """
query {
    project(fullPath: "PUT FULL PATH HERE") {
        issues(milestoneTitle:"PUT MILESTONE NAME HERE") {
            count
            edges {
                node {
                    iid
                    title
                    milestone {
                        title
                        startDate
                        dueDate
                    }
                    timelogs {
                        nodes {
                            id
                            spentAt
                            timeSpent
                            user {
                                name
                                username 
                            }
                        }    
                    }
                }
            }
        }
    }
}
"""
import requests
#make a request with the requests library to query the endpoint
request = requests.post(
    url=GITLAB_API_BASE_URL,
    json={"query": test_query},  
    headers={"Authorization": f"Bearer {token}"}
)
#Store the request json into a response variable for accessing the data
response = request.json()

#Number indexed is accessing a single issue.
issues_ = response["data"]["project"]["issues"]["edges"]
#specify start date of the milestone
start_date = response["data"]["project"]["issues"]["edges"][0]["node"]["milestone"]["startDate"]
due_date = response["data"]["project"]["issues"]["edges"][0]["node"]["milestone"]["dueDate"]





#Create a total array that will store the dictionaries for generating a markdown table
total_arr = []
users = []
#Iterate through the issues of the queried milestone 
for issue in issues_:
    #Store the title of the issue and the timelogs array
    title = issue['node']['title']
    iid = issue['node']['iid']
    timelogs = issue['node']['timelogs']['nodes']
    # print(timelogs)
    for time in timelogs:
        #Create a list of users by appending a new username when one is found
        if time['user']['name'] not in users:
            users.append(time['user']['name'])
    #Append dictionaries of title of the issue with empty string as the key in common
    total_arr.append({"" : "#" + iid + " " + title})
#sort the users list alphabetically by the members' last names
users.sort(key=lambda x:x.split()[-1])



#Append the dividers and the total time dictionary as well
total_arr.append({'' : "----"})
total_arr.append({'' : "----"})
total_arr.append({'' : "Total Time"})


In [ ]:
#Method to check if the spentAt date is earlier than the milestone start date
#param: date - the split list of the date the timelog is from
#param: start - the split list of the start date of the milestone
def earlier(date, start):
    #parse the passed in spentAt and start dates into ints to be compared
    date_parsed = list(map(lambda d : int(d), date))
    start_parsed = list(map(lambda m : int(m), start))
    earlier = False
    if date_parsed[0] < start_parsed[0]:
        earlier = True
    elif date_parsed[1] < start_parsed[1]:
        earlier = True
    elif date_parsed[1] == start_parsed[1]:
        if date_parsed[2] < start_parsed[2]:
            earlier = True
    return earlier

#Method to check if the spentAt date is later than the milestone due date
#param: date - the split list of the date the timelog is from
#param: start - the split list of the due date of the milestone
def later(date, due):
    #parse the passed in spentAt and start dates into ints to be compared
    date_parsed = list(map(lambda d : int(d), date))
    due_parsed = list(map(lambda m : int(m), due))
    later = False
    if date_parsed[0] > due_parsed[0]:
        later = True
    elif date_parsed[1] > due_parsed[1]:
        later = True
    elif date_parsed[1] == due_parsed[1]:
        if date_parsed[2] > due_parsed[2]:
            later = True
    return later    
    
    
#Method that splits the spentDate of the timelog, the start date of the milestone
#and the due date of the milestone
def date_in_range(spentDate : str):
    in_range = True
    split_spent = spentDate.split("-")
    split_spent[2] = split_spent[2][0:split_spent[2].index('T')]
    split_start = start_date.split("-")
    split_due = due_date.split("-")
    if earlier(split_spent, split_start) or later(split_spent, split_due):
        in_range = False   
    
    return in_range


#Function to get the time by user of each user for the specified PBI. 
#issueNum is the specified issue number to access for the correct timelogs in issues_
def time_by_issue(issueNum : int):
    spent_time = []
    #Create a tuple of the username and value for time (starts at 0)
    for user in users:
        
        spent_time.append([user,0])
    issueInfo = issues_[issueNum]['node']['timelogs']['nodes']
    #Iterate through each node of timelogs to store the time spent in hours for the correct username
    for node in issueInfo:
        user = node['user']['name']
        
        timeSpent = round(node['timeSpent'] /60 /60, 1)
        spentDate = node['spentAt']
        if date_in_range(spentDate):
            for x in range(len(spent_time)):
                if user in spent_time[x]:
                    spent_time[x][1] += timeSpent
    
    return spent_time
    

In [ ]:
#Add a value of 0 for each user in the Totals dictionary and add divider string to divider dictionary
for user in users:
    total_arr[len(total_arr) - 1][user] = 0
    total_arr[len(total_arr) - 2][user] = "----"
    total_arr[len(total_arr) - 3][user] = "----"
#iterate through each issue of the issues_ array to get the time by PBI
for i in range(len(issues_)):
    pbi_time = time_by_issue(i)
    #Iterate through the array of tuples of username and time spent
    for k in range(len(pbi_time)):
        #Store time by user in the correct issue dictionary
        total_arr[i][pbi_time[k][0]] = pbi_time[k][1]
        #Add time to the Totals dictionary
        total_arr[len(total_arr) - 1][pbi_time[k][0]] += pbi_time[k][1]

#The code below inserts the ceremonies time into the proper divided part of the table
#while also deleting the original ceremonies time
ceremonies = {}
for j in range(len(total_arr)):
    if "Ceremonies" in total_arr[j][""]:
        ceremonies[''] = "Ceremonies"
        for l in total_arr[j].keys():
            if type(total_arr[j][l]) != str:
                ceremonies[l] = round(total_arr[j][l], 1)
        total_arr.remove(total_arr[j])
        total_arr.insert(len(total_arr) - 2, ceremonies)
        break

markdown_query = markdown_table(total_arr).set_params(float_rounding=2,padding_width = 2, quote=False, row_sep = 'markdown').get_markdown()



